In [ ]:
# 针对DRA的来做一下实验

In [ ]:
# 提取数据
import pandas as pd

# cifar10 layer2
# fisher
fisher_data_path = '/home/dengruijun/data/project/AISecurity/Inverse_efficacy/results/2-3-VGG5/quantification/1/6.2/dFIL.csv'
# mse [euc,mse,ssim]
mse_data_path = '/home/dengruijun/data/FinTech/PP-Split/results/inverse-model-results-20240414/VGG5/1/layer2/inv-sim.csv'

# inv_data_path = '/home/dengruijun/data/FinTech/PP-Split/results/Purchase/1/layer3/inv-X.csv'
# sim_data_path = '/home/dengruijun/data/FinTech/PP-Split/results/Purchase/1/layer3/inv-sim.csv'

# raw_df = pd.read_csv(raw_data_path,sep=',',quotechar='"')
# inv_df = pd.read_csv(inv_data_path,sep=',',quotechar='"')
sim_df = pd.read_csv(mse_data_path,sep=',',quotechar='"')

# 打印文件基本信息
print('inv_sim_columns: ', sim_df.columns)


In [ ]:
# 提取出cos<0.4和cos>0.6的样本，生成两个df 
# 创建新的空DataFrame，用于存储满足条件的行  
# purchase
# df_low = inv_df[sim_df['cos'] < 0.6]  
# df_high = inv_df[sim_df['cos'] > 0.8]  
# df_low = inv_df[sim_df['mse'] < 0.16]  
# df_high = inv_df[sim_df['mse'] > 0.16]  
df_low = inv_df[sim_df['mse'] < 0.04]  
df_high = inv_df[sim_df['mse'] > 0.04]  

# # credit
# df_low = inv_df[sim_df['cos'] < 0.95]  
# df_high = inv_df[sim_df['cos'] > 0.998]  
# df_low = inv_df[sim_df['mse'] < 0.03]  
# df_high = inv_df[sim_df['mse'] > 0.03]  

# bank
# df_low = inv_df[sim_df['cos'] < 0.85]  
# df_high = inv_df[sim_df['cos'] > 0.95]  


# 打印结果  
print("df1中cos值小于的样本:")  
print(df_low.shape)  
print("\n df1中cos值大于的样本:")  
print(df_high.shape)


In [ ]:
# 计算这两类样本的平均熵
array_low = df_low.to_numpy()
array_high = df_high.to_numpy()
ent_list_lowcos = [entropy_prob(line) for line in array_low]
ent_list_highcos = [entropy_prob(line) for line in array_high]

print(len(ent_list_lowcos))
print(len(ent_list_highcos))
print(f"average low cos entropy: ", sum(ent_list_lowcos)/len(ent_list_lowcos))
print(f"average high cos entropy: ", sum(ent_list_highcos)/len(ent_list_highcos))

# 画图
import matplotlib.pyplot as plt  
counts_high, buckets_high = np.histogram(ent_list_highcos, bins=100, density=True) 
counts_low, buckets_low = np.histogram(ent_list_lowcos, bins=100, density=True) 
counts_high = counts_high/np.sum(counts_high)
counts_low = counts_low/np.sum(counts_low)

plt.rcParams.update({
#     'font.family': 'Times New Roman',
    'font.size': 22,
    'axes.labelsize': 22,
    'axes.titlesize': 24,
    'axes.linewidth': 1,
    'xtick.labelsize': 18,
    'ytick.labelsize': 18,
    'xtick.major.size': 18,
    'ytick.major.size': 18,
    'xtick.major.width': 1,
    'ytick.major.width': 1,
    'legend.fontsize': 18,
    'lines.linewidth': 40,
    'lines.markersize': 16,
    'axes.formatter.limits':(-2,2),
    'axes.formatter.useoffset':False,
    'axes.formatter.min_exponent':-2,
    'axes.grid': True,
    'grid.alpha': 0.53,
    'grid.linestyle': '--',
    # 'axes.spines.top': True,  # 修改此行
    # 'axes.spines.right': True  # 修改此行
})


plt.figure(figsize=(5,5))
plt.stairs(counts_low,buckets_low,fill=True, alpha=0.5)
plt.stairs(counts_high,buckets_high,fill=True, alpha=0.5)
# plt.scatter(range(len(ent_list_lowcos)), ent_list_lowcos, alpha=0.5)
# plt.scatter(range(len(ent_list_highcos)), ent_list_highcos, alpha=0.5)
plt.legend(['low-similarity','high-similarity'])
plt.ylabel('Fraction',fontsize=18)
plt.xlabel('Entropy of reconstr. input',fontsize=18)

plt.tight_layout()
# plt.savefig(f'./analysis_results/Purchase_entropy_distinct_histogram.pdf', dpi=300, format='pdf',bbox_inches ='tight')
# plt.savefig(f'./analysis_results/Credit_entropy_distinct_histogram.pdf', dpi=300, format='pdf',bbox_inches ='tight')
# plt.savefig(f'./analysis_results/Marketing_entropy_distinct_histogram.pdf', dpi=300, format='pdf',bbox_inches ='tight')
plt.show()
# 得出的结论是，cos越高，entropy越低？为什么？——这取决于原始数据的先验分布。